<a href="https://colab.research.google.com/github/Asato4931/Asato4931-CA_5_1-STS-Task-JSONL/blob/main/Re_5_1_STS_Task_%2B_JSON_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from transformers import AutoTokenizer

from sentence_transformers import SentenceTransformer
from sentence_transformers import models
from sentence_transformers import util

import numpy as np

import pandas as pd


from scipy import stats

import json

# Load pre-trained tokenizer
word_embedding_model = models.Transformer("cl-tohoku/bert-base-japanese-v2")
pooling = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True, pooling_mode_cls_token=False, pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling])

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")

#Sentence Embedding u,vをこれで出力、次元は768#

valid_data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/valid-v1.1.json', lines=True)


valid_data = valid_data.loc[:,['sentence1','sentence2', 'label']]

sen_no = valid_data.shape[0]

labels = []
cos_sims = []


#BERTを通してToken Embeddingを取得、Mean Pooling レイヤを通すことでSentence Embeddingを取得
for i in range(0,sen_no):
  emb_1 = model.encode(valid_data.loc[i,'sentence1'])
  emb_2 = model.encode(valid_data.loc[i,'sentence2'])
  label = valid_data.loc[i,'label']

#Sentence Embeddingからcos類似度を算出。この時点でCos類似度は0～1なので5倍にしてScaling
  cos_sim = 5 * util.cos_sim(emb_1, emb_2)
  cos_sims.append(cos_sim)
  labels.append(label)

#Pearson/Spearman係数を算出するために形を整える
cos_sims_arr = np.concatenate(cos_sims, axis=0)
cos_sims_arr_1d = cos_sims_arr.flatten()
labels_arr_1d = np.array(labels)


#得たCos類似度からPearson/Spearman係数を算出
pearson , _ = stats.pearsonr(cos_sims_arr_1d,labels_arr_1d)
spearman , _ = stats.spearmanr(cos_sims_arr_1d,labels_arr_1d)


#結果をJSONLにて出力
pearson_result = {"metrics" : 'Pearson' ,  "score" : pearson}
spearman_result = { "metrics" : 'Spearman' ,  "score" : spearman}

with open('/content/drive/MyDrive/Colab Notebooks/CA課題用/CA課題5/Re CA_5_1 STS Task Results.jsonl' , 'w') as file:
  file.write(json.dumps(pearson_result) + '\n')
  file.write(json.dumps(spearman_result) + '\n')


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!pip install -U sentence-transformers

!apt-get install mecab mecab-ipadic-utf8 python-mecab libmecab-dev
!pip install mecab-python3

!pip install fugashi

!pip install unidic_lite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=83509d60d8860bb48bbe2d09769eb9f134f206bc9748c706af880ee029d8d47b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Reading package lists... Done
Building dependency tree